# Toronto Neighborhood Data Project

## Scope of the Project

The idea is to follow a similar neighborhood clustering analysis that we performed for Manhattan (in the NYC_Neighborhood_Segmentation notebook). However, there are a few challenges we need to address.

The neighborhood data for New York was provided in an easily accessible json file for us to use. In this case we will be using Beautiful Soup to scrape the Toronto neighborhood data from Wikipedia.  Additionally, since we are scraping web data to get our neighborhoods, there will be a lot more data cleaning and formatting work that we need to do before we can start any sort of analysis.

### Data Scraping

Here we use the Beautiful Soup library to get data from web pages (in particular, Wikipedia pages on Toronto).

The Wikipedia page we are using as the basis for our Toronto neighborhoods can be found <a href="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M">https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M</a>.

In [17]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

from geopy.geocoders import Nominatim
import folium

In [18]:
# Use the requests package to get the postal code page from Wikipedia

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url).text

# Use beautiful soup to get a cleaned up html file of the page, and to extract the table from the file
post_code_page = BeautifulSoup(page, 'html.parser')
post_code_table = post_code_page.find('table')

# Use the read_html function in pandas to create a dataframe from the table
post_code_df = pd.read_html(str(post_code_table))[0]

# Drop postal codes where Borough is "Not assigned"
post_code_df = post_code_df[post_code_df['Borough'] != "Not assigned"]

# Check that the dataframe is as expected
post_code_df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


We now have a pandas dataframe with the Toronto postal codes as well as the borough and neighborhoods that are associated with each postal code.  We removed "Not Assigned" postal codes.

Finally, we check the shape of the dataframe to make sure we have the number of rows that we expect.

In [19]:
post_code_df.shape

(103, 3)

## Adding Geo Coordinates to the Data Frame

First, we need to get latitude and longitude for each of the postal codes.  We load a file, **Geospatial_Coordinates.csv** that has this information. Currently the Google API for accessing this data is no longer free. There is a free alternative, geocoder, but it does not seem to provide consistent or reliable data.

In [20]:
# Read the csv file into a pandas dataframe
geo_coordinates = pd.read_csv('Geospatial_Coordinates.csv')

# Check the data
geo_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
# Merge the two data frames based on postal code

post_code_coordinates = post_code_df.merge(geo_coordinates, how='left', on='Postal Code')

post_code_coordinates.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Exploring and Clustering Neighborhoods in the Toronto Area

Finally, we use the foursquare API and folium to explore, cluster and visualize neighborhoods in the Toronto area.

First, we use geocoders to get the latitude and longitude of Toronto and then build a map of the area with Folium.

In [22]:
# Get coordinates for Toronto, Ontario
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of Toronto are {}, {}.".format(latitude, longitude))

The geographical coordinates of Toronto are 43.6534817, -79.3839347.


In [26]:
# Use folium to create a map of the Toronto

map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

for lat, lng, borough, neighbourhood in zip(post_code_coordinates['Latitude'], post_code_coordinates['Longitude'], post_code_coordinates['Borough'], post_code_coordinates['Neighbourhood']):
    label = '{},{}'.format(neighbourhood, borough)
    label = folium.Popup(label,parse_html = True)
    folium.CircleMarker(
        [lat,lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_toronto)
map_toronto